In [1]:
! pip install -qU "langchain[mistralai]"

In [2]:
import getpass
import os

if not os.environ.get("MISTRAL_API_KEY"):
  os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter API key for Mistral AI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("mistral-large-latest", model_provider="mistralai")


In [3]:
! pip install -qU langchain-mistralai

In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

# Access the HF_TOKEN
hf_token = os.getenv("HF_TOKEN")


In [5]:
! pip install transformers huggingface_hub


In [8]:
import os

os.environ["HF_TOKEN"] = hf_token


In [ ]:
from huggingface_hub import login

login(hf_token)


/Users/nirajanpaudel17/anaconda3/envs/learn_crew/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [9]:
import getpass
import os

if not os.environ.get("MISTRALAI_API_KEY"):
  os.environ["MISTRALAI_API_KEY"] = getpass.getpass("Enter API key for MistralAI: ")

from langchain_mistralai import MistralAIEmbeddings

embeddings = MistralAIEmbeddings(model="mistral-embed")

/Users/nirajanpaudel17/anaconda3/envs/learn_crew/lib/python3.10/site-packages/langchain_mistralai/embeddings.py:181: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [10]:
! pip install -qU langchain-core

In [11]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [12]:
! capture --no-stderr
! pip install --upgrade --quiet langgraph langchain-community beautifulsoup4

zsh:1: command not found: capture


In [14]:
os.environ["LANGSMITH_TRACING"] = "true"
if not os.environ.get("LANGSMITH_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [16]:
import bs4
from langchain import hub
from langchain_community.document_loaders import TextLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

loader = TextLoader("./sample.txt")

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

In [17]:
# Index chunks
_ = vector_store.add_documents(documents=all_splits)

In [18]:
from langgraph.graph import MessagesState, StateGraph

graph_builder = StateGraph(MessagesState)

In [19]:
from langchain_core.tools import tool


@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = vector_store.similarity_search(query, k=2)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\n" f"Content: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

In [20]:
from langchain_core.messages import SystemMessage
from langgraph.prebuilt import ToolNode


# Step 1: Generate an AIMessage that may include a tool-call to be sent.
def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}


# Step 2: Execute the retrieval.
tools = ToolNode([retrieve])


# Step 3: Generate a response using the retrieved content.
def generate(state: MessagesState):
    """Generate answer."""
    # Get generated ToolMessages
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]

    # Format into prompt
    docs_content = "\n\n".join(doc.content for doc in tool_messages)
    system_message_content = (
        "You are an assistant for question-answering tasks. "
        "Use the following pieces of retrieved context to answer "
        "the question. If you don't know the answer, say that you "
        "don't know. Use three sentences maximum and keep the "
        "answer concise."
        "\n\n"
        f"{docs_content}"
    )
    conversation_messages = [
        message
        for message in state["messages"]
        if message.type in ("human", "system")
        or (message.type == "ai" and not message.tool_calls)
    ]
    prompt = [SystemMessage(system_message_content)] + conversation_messages

    # Run
    response = llm.invoke(prompt)
    return {"messages": [response]}

In [21]:
from langgraph.graph import END
from langgraph.prebuilt import ToolNode, tools_condition

graph_builder.add_node(query_or_respond)
graph_builder.add_node(tools)
graph_builder.add_node(generate)

graph_builder.set_entry_point("query_or_respond")
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()

In [31]:
! pip install --upgrade pydantic

In [ ]:
input_message = "Hello"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
):
    # print(step) 
    step["messages"][-1].pretty_print()


{'messages': [HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}, id='0ad46754-da8f-4685-8547-fc7ac0c4e809')]}
================================ Human Message =================================

Hello
{'messages': [HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}, id='0ad46754-da8f-4685-8547-fc7ac0c4e809'), AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'prompt_tokens': 65, 'total_tokens': 74, 'completion_tokens': 9}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-2ce269f2-ce77-494d-9b87-115a0bdf867b-0', usage_metadata={'input_tokens': 65, 'output_tokens': 9, 'total_tokens': 74})]}
================================== Ai Message ==================================

Hello! How can I assist you today?


In [23]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

# Specify an ID for the thread
config = {"configurable": {"thread_id": "abc123"}}

In [24]:
input_message = "What is venv?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is venv?
================================== Ai Message ==================================

`venv` is a module in Python that provides support for creating lightweight "virtual environments" with their own site directories, optionally isolated from system site directories. Each virtual environment has its own Python binary (which matches the version of the binary that was used to create this environment) and can have its own independent set of installed Python packages.

### Key Features of `venv`:
1. **Isolation**: It allows you to isolate your project dependencies from other projects. This means you can work on multiple projects with different dependencies without them interfering with each other.
2. **Reproducibility**: You can easily recreate the same environment on another machine by sharing the requirements file (`requirements.txt`).
3. **Dependency Management**: You can manage and install packa

In [25]:
input_message = "can you clarify step 2?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

can you clarify step 2?
================================== Ai Message ==================================

Certainly! Step 2 involves activating the virtual environment you created in Step 1. Activating a virtual environment means configuring your shell to use the Python interpreter and packages from the virtual environment instead of the global Python installation.

Here's a detailed explanation of how to activate a virtual environment:

### Activating a Virtual Environment

1. **On Windows**:
   - Open your command prompt or PowerShell.
   - Navigate to the directory where your virtual environment is located.
   - Run the following command to activate the virtual environment:
     ```sh
     myenv\Scripts\activate
     ```
   - After activation, your command prompt will change to show the name of the activated environment, like this:
     ```
     (myenv) C:\path\to\your\project>
     ```

2. **On Unix o

In [26]:
input_message = "can you tell me more about steps on Unix or macos?"

for step in graph.stream(
    {"messages": [{"role": "user", "content": input_message}]},
    stream_mode="values",
    config=config,
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

can you tell me more about steps on Unix or macos?
================================== Ai Message ==================================
Tool Calls:
  retrieve (bq8PzpkXz)
 Call ID: bq8PzpkXz
  Args:
    query: steps to activate virtual environment in Unix or MacOS
================================= Tool Message =================================
Name: retrieve

Source: {'source': './sample.txt'}
Content: A virtual environment may be “activated” using a script in its binary directory (bin on POSIX; Scripts on Windows). This will prepend that directory to your PATH, so that running python will invoke the environment’s Python interpreter and you can run installed scripts without having to use their full path. The invocation of the activation script is platform-specific (<venv> must be replaced by the path to the directory containing the virtual environment):

Platform

Shell

Command to activate virtual environmen

In [28]:
# input_message = "what is the pdf about?"

# for step in graph.stream(
#     {"messages": [{"role": "user", "content": input_message}]},
#     stream_mode="values",
#     config=config,
# ):
#     step["messages"][-1].pretty_print()